In [1]:
import numpy as np
import pandas as pd

corpus = pd.read_csv('definizioni.csv', sep=',', engine='python')
corpus


,Unnamed: 0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P23,P24,P25,P27,P29,P30,P31,P34,P35,GPT3
0,Emotion,Range of concepts human beings feel in certain...,Something you can feel,Something that an animal can feel,something you think that makes you feel good o...,Human sensation arising from the form of feelings,State of mind that a living being can percieve,Feeling that a human or an animal can express ...,A sentiment that a living entity can feel and ...,what you feel in a certain moment,...,feeling of a living being,feeling,A particular state of mind of a person caused ...,A mental reaction,a strong feeling,Feeling/State felt by living beings,The mental state of an agent,Mental mood,a feeling dued to something,Emotion is a feeling that is created by a phys...
1,Person,Human being,Human being,"The generic for person is human, it describe a...",human,Living human being belonging to a group or soc...,Mammal descending from apes,"Living entity, human being, sentient.",A human being.,human being u can see or touch,...,human,human being,An individual of the homo sapiens species,A human being,a human being,"Human being, man or woman",A specific human being.,Human being,a living entity,There is no single answer to this question as ...
2,Revenge,"Feeling, usually negative. Consequence of a wr...",An emotion that describes anger,"Is classified as an emotion, generally not goo...",negative emotion,Reaction of anger or violence on oneself or on...,Act performed by a living being triggered by e...,Act or feeling of anger resulting from another...,Act of doing something to someone because of a...,"action against something or someone, generally...",...,emotion caused by and towards someone,angry reaction towards someone,A person's will to get back at someone for som...,The mental reaction of getting something to ma...,the act of damaging someone in return of an in...,The wish to damage someone who hurt you.,The act of damaging someone as a reaction,Mood,desire to do justice for themselves,Revenge is a type of justice that is often see...
3,Brick,"object made of a material (e.g. clay), usually...","block of some material, used in construction",It’s an object and is the basic element of a c...,piece of a material used to build something,Parallelepiped object used for the constructio...,Construction tool of resistnat material and wi...,"Material used for construction of buildings, c...",An object used to build walls.,Red object used to build constructions,...,material made of clay used to build a structure,piece of material used to build something,An object usually used for constructing buildi...,A rectangular-shaped object made of clay used ...,material used for construction,NaN,Material used for construction.,An object made to build something,object used to build something,A brick is a rectangular block of ceramic mate...


Per ogni concetto, prendere tutte le definizioni, e calcolare la sovrapposizione lessicale tra tutte le definizioni (cioè le parole in comune). E' possibile che per un dato concetto l'intersezione delle parole sia vuota. Ad esempio, se una definizione contiene una parola che non viene usata dalle altre non verrà conteggiata. Un modo è contare le parole più ricorrenti (contando la loro frequenza) e poi vedere quanto quelle parole ci sono nelle definizioni (es. 12/30 usano una certa parola) e da qui calcoliamo uno score delle parole

Una volta definita la metrica per la sovrapposizione lessicale, aggreghiamo le dimensioni proposte. Cioè, prendiamo i due concetti concreti e vediamo com'è l'andamento delle parole, e idem per quelli astratti. Faremo la stessa cosa per quelli generici e quelli specifici.

Ciò viene fatto per vedere se ci sono differenze tra le dimensioni. Tipicamente, c'è molta più sovrapposizione su concetti concreti e su concetti specifici